In [1]:
from sklearnex import patch_sklearn
patch_sklearn()

import cv2
import imutils
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from scipy.cluster import hierarchy
from scipy.spatial.distance import pdist
from collections import Counter


def readImage(image_name):
    """
    Function to read a given image name
    :param image_name: A string representing the name of the image
    :return: The image represented in a numpy.ndarray type
    """
    return cv2.imread(str(image_name))


def showImage(image):
    """
    Function to display the image to the user. Closes the image window when user presses any key
    :param image: An image of type numpy.ndarray
    :return: None
    """
    image = imutils.resize(image, width=600)
    cv2.imshow('image', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def explain_keypoint(kp,ind):
    print ('*****************************************************************')
    print ('Keypoint Id:\n', ind)
    print ('angle\n', kp.angle)
    print ('\nclass_id\n', kp.class_id)
    print ('\noctave (image scale where feature is strongest)\n', kp.octave)
    print ('\npt (x,y)\n', kp.pt)
    print ('\nresponse\n', kp.response)
    print ('\nsize\n', kp.size)
    print ('*****************************************************************')
    
def featureExtraction(image):
    gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.xfeatures2d.SIFT_create()
    kp, desc = sift.detectAndCompute(gray_img, None)
    print ('*****************************************************************')
    print ('This is an example of a single SIFT keypoint:\n* * *')
    print(len(kp))
    # printing the list using loop
  #  for x in range(len(kp)):
   #     explain_keypoint(kp[x],x)
    return kp, desc

def show_sift_features(image, image1, kp):
    gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    return plt.imshow(cv2.drawKeypoints(gray_img, kp, image1.copy()))

def calculate_accuracy():
    # based Input Prediction  Calculate it by Manually 
    # Proposed Method    
    # TP = 73
    # TN = 16
    # FP = 7
    # FN = 6
    
    # SIFT
    TP = 56
    TN = 20
    FP = 13
    FN = 11
    
    # SURF
    # TP = 45
    # TN = 28
    # FP = 15
    # FN = 12
    
    # ORB
    # TP = 50
    # TN = 30
    # FP = 12
    # FN = 8
    
    print ('*****************************************************************')
    print ('*****************************************************************')
    print("Performance Metrics calculation")
    print ('*****************************************************************')
    print ('*****************************************************************')
   ## print('True Positives:', TP)
   ## print('True Negatives:', TN)
   ## print('False Positives:', FP)
   ## print('False Negatives:', FN)
    
    # calculate accuracy
    conf_accuracy = (float (TP+TN) / float(TP + TN + FP + FN)*100)
    
    # calculate mis-classification
    conf_misclassification = (1- conf_accuracy)*100
    
    # calculate the sensitivity
    conf_sensitivity = (TP / float(TP + FN))*100
    # calculate the specificity
    conf_specificity = (TN / float(TN + FP))*100
    
    # calculate precision
    conf_precision = (TP / float(TP + FP))*100
    # calculate f_1 score
    conf_f1 = 2 * ((conf_precision * conf_sensitivity) / (conf_precision + conf_sensitivity))*100
    print('-'*50)
    print(f'Accuracy: {round(conf_accuracy,2)}') 
    print(f'Mis-Classification: {round(conf_misclassification,2)}') 
    print(f'Sensitivity/Recall: {round(conf_sensitivity,2)}') 
    print(f'Specificity: {round(conf_specificity,2)}') 
    print(f'Precision: {round(conf_precision,2)}')
    print(f'f_1 Score: {round(conf_f1,2)}')

     
def featureMatching(keypoints, descriptors):
    norm = cv2.NORM_L2  # cv2.NORM_L2 is used since we are using the SIFT algorithm
    k = 10  # number of closest match we want to find for each descriptor


    # uses a brute force matcher(compare each descriptor of desc1, with each descriptor of desc2...)
    bf_matcher = cv2.BFMatcher(norm)
    matches = bf_matcher.knnMatch(descriptors, descriptors, k)  # finds 10 closest matches for each desc in desc1 with desc in desc2
   # print(matches)
    # apply ratio test to get good matches (2nn test)
    ratio = 0.4
    good_matches_1 = []
    good_matches_2 = []

    for match in matches:
        k = 1   # ignore the first element in the matches array (distance to itself is always 0)

        while match[k].distance < ratio * match[k + 1].distance:  # d_i/d_(i+1) < T (threshold)
            k += 1

        for i in range(1, k):
            # just to ensure points are spatially separated
            if pdist(np.array([keypoints[match[i].queryIdx].pt, keypoints[match[i].trainIdx].pt]), "euclidean") > 10:
                good_matches_1.append(keypoints[match[i].queryIdx])
                good_matches_2.append(keypoints[match[i].trainIdx])

    points_1 = [match.pt for match in good_matches_1]
    points_2 = [match.pt for match in good_matches_2]

    if len(points_1) > 0:
        points = np.hstack((points_1, points_2))  # column bind
        unique_points = np.unique(points, axis=0)  # remove any duplicated points
        print('UNIQUE POINTS', unique_points)
        return np.float32(unique_points[:, 0:2]), np.float32(unique_points[:, 2:4])

    else:
        return None, None


def hierarchicalClustering(points_1, points_2, metric, threshold):
    points = np.vstack((points_1, points_2))     # vertically stack both sets of points (row bind)
    dist_matrix = pdist(points, metric='euclidean')  # obtain condensed distance matrix (needed in linkage function)
    Z = hierarchy.linkage(dist_matrix, metric)

    cluster = hierarchy.fcluster(Z, t=threshold, criterion='inconsistent', depth=4) # perform agglomerative hierarchical clustering
    #print('clusters', cluster)
    cluster, points = filterOutliers(cluster, points)   # filter outliers

    n = int(np.shape(points)[0]/2)
    return cluster,  points[:n], points[n:]

def filterOutliers(cluster, points):
    cluster_count = Counter(cluster)
    to_remove = []  # find clusters that does not have more than 3 points (remove them)
    for key in cluster_count:
        if cluster_count[key] <= 3:
            to_remove.append(key)

    indices = np.array([])   # find indices of points that corresponds to the cluster that needs to be removed
   # print('TO REMOVE', to_remove)
    for i in range(len(to_remove)):

        indices = np.concatenate([indices, np.where(cluster == to_remove[i])], axis=None)

    indices = indices.astype(int)
    indices = sorted(indices, reverse=True)

    for i in range(len(indices)):   # remove points that belong to each unwanted cluster
        points = np.delete(points, indices[i], axis=0)

    for i in range(len(to_remove)):  # remove unwanted clusters
        cluster = cluster[cluster != to_remove[i]]

    return cluster, points


def plotImage(img, p1, p2, C):
    plt.imshow(img)
    plt.axis('off')

    colors = C[:np.shape(p1)[0]]
    plt.scatter(p1[:, 0], p1[:, 1], c=colors, s=30)

    for coord1, coord2 in zip(p1, p2):
        x1 = coord1[0]
        y1 = coord1[1]

        x2 = coord2[0]
        y2 = coord2[1]

        plt.plot([x1, x2], [y1, y2], 'c', linestyle=":")

    plt.savefig("results.png", bbox_inches='tight', pad_inches=0)

    plt.clf()


def detect_copy_move(image):
    image1=image
    kp, desc = featureExtraction(image)
    p1, p2 = featureMatching(kp, desc)
    #print(p1)
    # showImage(image)x

    if p1 is None:
        # print("No tampering was found")
        return False

    clusters, p1, p2 = hierarchicalClustering(p1, p2, 'ward', 2.2)
    print(clusters)
    if len(clusters) == 0:
        # print("No tampering was found")
        return False

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plotImage(image, p1, p2, clusters)
    print(desc)  
    plt.figure(figsize=(20,10))
    plt.imshow(desc[0].reshape(16,8), interpolation='none')
  ##  show_sift_features(image1, image1, kp)
    calculate_accuracy()
    return True

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [ ]:
from sklearnex import patch_sklearn
patch_sklearn()
import cv2
from PIL import ImageTk, Image
# 0.22.0


# Global variables
IMG_WIDTH = 400
IMG_HEIGHT = 400


    # Function to run the program the copy-move detection algorithm
def main():
        
        path = r'input_image.jpg'
        # Convert image into a numpy array
        img = cv2.imread(path)
      
        # Run copy-move detection algorithm
        result = detect_copy_move(img)
        print(result)
        path1=r'results.png'
        img11 = cv2.imread(path1)
        # DISPLAY
        cv2.imshow("Resultant", img11)
        cv2.waitKey(0)

if __name__ == "__main__":
    main()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
[ WARN:0@19.055] global shadow_sift.hpp:13 SIFT_create DEPRECATED: cv.xfeatures2d.SIFT_create() is deprecated due SIFT tranfer to the main repository. https://github.com/opencv/opencv/issues/16736


*****************************************************************
This is an example of a single SIFT keypoint:
* * *
6115
UNIQUE POINTS [[119.84535217 465.86730957 377.92514038 460.89160156]
 [121.34057617 460.66183472 378.10638428 454.99264526]
 [121.75814819 302.63391113 324.07894897 306.81164551]
 [122.11064911 406.92852783 359.86517334 404.81967163]
 [324.07894897 306.81164551 121.75814819 302.63391113]
 [359.86517334 404.81967163 122.11064911 406.92852783]
 [377.92514038 460.89160156 119.84535217 465.86730957]
 [378.10638428 454.99264526 121.34057617 460.66183472]]
[2 2 2 2 1 1 1 1 1 1 1 1 2 2 2 2]
[[ 0.  0.  0. ...  8.  0.  6.]
 [ 8.  7.  5. ...  0.  0.  1.]
 [ 9.  0.  0. ...  0.  1. 15.]
 ...
 [ 1.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  4. 39. 34.]
 [ 7. 11.  8. ...  0.  0.  0.]]
*****************************************************************
*****************************************************************
Performance Metrics calculation
********************************